In [1]:
import datasets
import orjson
from tqdm.auto import tqdm
from pathlib import Path

paths = Path("/mnt/harddrive/datasets/")

print([p.name for p in paths.glob("*")])

['finepdfs_en_2020plus_100b', 'data', 'marin-arxiv-3b', 'README.md', 'download_dclm_edu.py', 'marin-wikipedia-markdown-8b', 'marin-datashop-science-qa-12b', '.gitattributes', 'zyda-2-100b', 'opc-fineweb-code-corpus', 'bigcode_the_stack_v2_updated_smol', 'marin-stackexchange-markdown-20b', 'finemath-4plus-10b', '.cache', 'logs', 'dclm-edu']


In [2]:
!uv pip install datasets orjson hf_transfer

Resolved 34 packages in 1.48s                                        
Uninstalled 3 packages in 3ms
Installed 3 packages in 3ms                                 
 - dill==0.4.0
 + dill==0.3.8
 - fsspec==2025.7.0
 + fsspec==2025.3.0
 - multiprocess==0.70.18
 + multiprocess==0.70.16


In [2]:
jsons = list(Path("/home/george/coding_parser/out/chunk-00").glob("*.hf.json"))

In [3]:
len(jsons)

431786

In [ ]:
dicts = [orjson.loads(fp.read_bytes()) for fp in tqdm(jsons[200000:])]

  0%|          | 0/231786 [00:00<?, ?it/s]

 16%|█▋        | 38238/231786 [00:21<01:20, 2404.92it/s]

In [2]:
import pandas as pd
import ast

# df = pd.concat([pd.read_csv("dicts_1.csv", header=0, converters={"files": ast.literal_eval}),
#                 pd.read_csv("dicts_2.csv", header=0, converters={"files": ast.literal_eval})])
df = pd.concat([pd.read_csv("dicts_1.csv", header=0), pd.read_csv("dicts_2.csv", header=0)])
df.head()

,Unnamed: 0,repo_name,repo_url,snapshot_id,revision_id,directory_id,branch_name,visit_date,revision_date,committer_date,github_id,star_events_count,fork_events_count,gha_license_id,gha_created_at,gha_updated_at,gha_pushed_at,gha_language,files,num_files
0,0,pericdario/remote-nunchuk-keyboard,https://github.com/pericdario/remote-nunchuk-k...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236235108,0,0,NaN,NaN,2020-01-26T00:20:14+00:00,2020-01-26T00:20:12+00:00,NaN,[{'blob_id': '776a2cd3d70dfc5307e2204ab6c5adc6...,8
1,1,w3c/wai-dynamic-planning,https://github.com/w3c/wai-dynamic-planning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29198001,5,0,NaN,NaN,2025-07-16T22:53:48+00:00,2025-07-16T22:53:19+00:00,NaN,[{'blob_id': '55bba6c86b12e7b6abeaf71b8d0053ef...,14
2,2,yihanYozikua/distance-measure,https://github.com/yihanYozikua/distance-measure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313556027,1,0,NaN,NaN,2021-06-23T12:41:45+00:00,2021-04-08T08:46:36+00:00,NaN,[{'blob_id': '47fe6ac1012d739c94ffaa6d4f0e1a13...,5
3,3,AuroransSolis/osu-db-manager,https://github.com/AuroransSolis/osu-db-manager,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184883096,1,0,NaN,NaN,2020-08-16T10:01:09+00:00,2020-08-16T10:01:07+00:00,NaN,[{'blob_id': 'e9a9f2431d9282dfcaeb278cd3529542...,46
4,4,lucasfloripa/Elciess,https://github.com/lucasfloripa/Elciess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252508866,1,0,NaN,NaN,2021-06-03T18:23:32+00:00,2022-12-22T15:17:33+00:00,NaN,[{'blob_id': 'ddacb0c25af94a57e2d98659a39b813c...,592


In [ ]:
# All repos from original dataset are parsed with Github API and re-downloaded,
# so respective updates are kept, metadata is updated. This took 10+ days to process due to GraphQL limits.

# Filtering rules
# - Removed repos with no update in the last 6 years (no updates since September 2019)
# - Removed files with a single line
# - Removed repos with a single file
# - Removed repos with more than 99% percentile number of files (3202)
# - Removed repos with abnormal alphanum factor (either all characters or all numbers)
# Those rules remove about 5.5% of repos

# In addition, during parsing I applied set of heuristics from the original paper, this includes:
# binary_or_size_check, length_checks, autogen_check, alpha_check, encoded_check and language specific filters


# All filtered files are removed from `files` list field, so repos might be not full

# In result, there are 408716 repos with a total of 30M files

In [26]:
import seaborn as sns

sns.set_theme()

In [ ]:
sum([len(f) for f in df["files"]])

15234863341

In [11]:
df["files"] = df["files"].apply(lambda x: [f for f in x if f["num_lines"] > 1])

# removes 4%
df = df[df.apply(lambda x: len(x.get("files", [])) > 1, axis=1)]
# removes 1%
df = df[df.num_files < 3202]
# df = df[df.apply(lambda x: all(len(line) < 240 for file in x.get('files', []) for line in file.get('lines', [])), axis=1)]
# df = df[df.apply(lambda x: all(len(line) > 120 for file in x.get('files', []) for line in file.get('lines', [])), axis=1)]

,repo_name,repo_url,snapshot_id,revision_id,directory_id,branch_name,visit_date,revision_date,committer_date,github_id,star_events_count,fork_events_count,gha_license_id,gha_created_at,gha_updated_at,gha_pushed_at,gha_language,files,num_files
0,raafvargas/gqlgen-demo,https://github.com/raafvargas/gqlgen-demo,,,,,None,None,None,376599692,1,0,,None,2021-06-18T23:31:59+00:00,2021-06-15T12:51:47+00:00,,[{'blob_id': 'c154856e505e6b80b8e24349c0e46fb4...,15
1,arthurdlima/Notes-App,https://github.com/arthurdlima/Notes-App,,,,,None,None,None,281958388,1,0,,2020-07-23T13:25:53+00:00,2020-08-13T03:23:02+00:00,2020-08-13T03:22:59+00:00,Hack,[{'blob_id': '44d905d2bdf3d365b00bad2b34e241ac...,8
2,mel-harris/cvml,https://github.com/mel-harris/cvml,,,,,None,None,None,236818544,1,0,,None,2020-03-11T18:24:02+00:00,2020-03-11T18:22:03+00:00,,[{'blob_id': '69dd3e7ba7f8215afe9b9e4e3d33df2b...,8
3,nikleechan/EmbyCrack,https://github.com/nikleechan/EmbyCrack,,,,,None,None,None,298985605,8,0,,None,2023-12-22T00:56:41+00:00,2020-09-23T13:53:28+00:00,,[{'blob_id': 'aa4bd4d0cf4a7d57936384ff696d7ef4...,303
4,Freifunk-Nord/nord-puppet-config,https://github.com/Freifunk-Nord/nord-puppet-c...,,,,,None,None,None,66150057,2,0,,None,2021-03-06T21:09:06+00:00,2021-03-06T21:09:03+00:00,,[{'blob_id': '00e722ec8e18a313cc9519eb81ea77cb...,5


In [ ]:
import numpy as np

np.concatenate(df.apply(lambda x: [f["alphanum_fraction"] for f in x["files"] if f["num_lines"] > 1], axis=1).values)

array([0.63510638, 0.62893082, 0.63367347, ..., 0.52951389, 0.61121157,
       0.58057851], shape=(1626417,))

In [ ]:
# removes 0.0% ,just safety check
np.concatenate(
    df.apply(
        lambda x: [
            f["alphanum_fraction"] for f in x["files"] if f["num_lines"] > 1 and 0.3 < f["alphanum_fraction"] < 0.9
        ],
        axis=1,
    ).values
)

array([0.63510638, 0.62893082, 0.63367347, ..., 0.52951389, 0.61121157,
       0.58057851], shape=(1624302,))

In [27]:
import numpy as np

sns.displot(
    np.concatenate(
        df.apply(lambda x: [f["alphanum_fraction"] for f in x["files"] if f["num_lines"] > 1], axis=1).values
    ),
)

TypeError: string indices must be integers, not 'str'

In [4]:
df

,Unnamed: 0,repo_name,repo_url,snapshot_id,revision_id,directory_id,branch_name,visit_date,revision_date,committer_date,github_id,star_events_count,fork_events_count,gha_license_id,gha_created_at,gha_updated_at,gha_pushed_at,gha_language,files,num_files
0,0,pericdario/remote-nunchuk-keyboard,https://github.com/pericdario/remote-nunchuk-k...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236235108,0,0,NaN,NaN,2020-01-26T00:20:14+00:00,2020-01-26T00:20:12+00:00,NaN,[{'blob_id': '776a2cd3d70dfc5307e2204ab6c5adc6...,8
1,1,w3c/wai-dynamic-planning,https://github.com/w3c/wai-dynamic-planning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29198001,5,0,NaN,NaN,2025-07-16T22:53:48+00:00,2025-07-16T22:53:19+00:00,NaN,[{'blob_id': '55bba6c86b12e7b6abeaf71b8d0053ef...,14
2,2,yihanYozikua/distance-measure,https://github.com/yihanYozikua/distance-measure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313556027,1,0,NaN,NaN,2021-06-23T12:41:45+00:00,2021-04-08T08:46:36+00:00,NaN,[{'blob_id': '47fe6ac1012d739c94ffaa6d4f0e1a13...,5
3,3,AuroransSolis/osu-db-manager,https://github.com/AuroransSolis/osu-db-manager,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184883096,1,0,NaN,NaN,2020-08-16T10:01:09+00:00,2020-08-16T10:01:07+00:00,NaN,[{'blob_id': 'e9a9f2431d9282dfcaeb278cd3529542...,46
4,4,lucasfloripa/Elciess,https://github.com/lucasfloripa/Elciess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252508866,1,0,NaN,NaN,2021-06-03T18:23:32+00:00,2022-12-22T15:17:33+00:00,NaN,[{'blob_id': 'ddacb0c25af94a57e2d98659a39b813c...,592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219272,231779,iannesbitt/CaseThwaitesSPH,https://github.com/iannesbitt/CaseThwaitesSPH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179379210,1,0,NaN,2019-04-03T22:24:16+00:00,2023-10-19T04:50:29+00:00,2020-11-03T15:21:06+00:00,Shell,[{'blob_id': 'bc45d66d73bcabfed892635f2be0d9df...,3
219273,231780,beikesea/dell-7820-fans,https://github.com/beikesea/dell-7820-fans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,191957143,2,0,NaN,NaN,2021-11-30T08:34:29+00:00,2019-11-05T16:25:55+00:00,NaN,[{'blob_id': 'b36f9eb3dc09cf352afb996567d0c2c3...,2
219274,231781,venuses/go-cmq,https://github.com/venuses/go-cmq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224149135,1,0,NaN,NaN,2022-08-19T11:46:24+00:00,2022-08-19T12:30:32+00:00,NaN,[{'blob_id': 'd9d444d94bfacc6a250f7f118b05df65...,16
219275,231784,harshsinghvi/c-cpp-examples-resources,https://github.com/harshsinghvi/c-cpp-examples...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292811105,2,0,NaN,2020-09-04T09:47:30+00:00,2021-04-29T11:37:12+00:00,2021-04-26T05:32:45+00:00,C++,[{'blob_id': '7d9e91b125d828c3761ad3d56d83744a...,21


In [5]:
next(iter(mirrors_dir.glob("*")))

PosixPath('/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/mirror/chunk-00/lang-do__j2w-lekce10.tar.gz')

In [7]:
chunk_dir = Path("/home/george/coding_parser/out/chunk-00")
mirrors_dir = Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/mirror/chunk-00")


all_repos = set([f"{item['repo_name'].replace('/', '__')}.tar.gz" for _, item in df.iterrows()])


def process_repo(item):
    files = item["files"]
    {f["path"] for f in files}
    # print(local_filenames)

    # fp = chunk_dir / f"{item['repo_name']}.hf.json"
    # d = orjson.loads(fp.read_bytes())
    f"{item['repo_name'].replace('/', '__')}.tar.gz"

In [22]:
sizes = [p.stat().st_size / (1024 * 1024) for p in mirrors_dir.glob("*")]

In [23]:
import numpy as np

np.percentile(sizes, 99)

np.float64(60.44981792449954)

In [21]:
for p in mirrors_dir.glob("*"):
    if p.stat().st_size / (1024 * 1024) > 96:
        p.unlink()

In [ ]:
# read some files with alphanum_fraction < 0.3
# break when found 10
files = []
for fp in Path("/home/george/coding_parser/out/chunk-00").glob("*.hf.json"):
    d = orjson.loads(fp.read_bytes())
    for f in d["files"]:
        if f["num_lines"] > 1 and f["alphanum_fraction"] < 0.3:
            files.append((f, fp.name))
        if len(files) == 10:
            break

[({'blob_id': 'cd783dbc34a72eca6390c4825383eb8149ebaf1a',
   'path': 'smoke/test/dig/dig.rb',
   'content_id': '7ea76dcfa9fdefda7fc74f29f233de77ad8d34ca246b7a24efeb05fa41b3f091',
   'language': 'Ruby',
   'length_bytes': 84,
   'detected_licenses': [],
   'license_type': '',
   'src_encoding': 'utf-8',
   'is_vendor': False,
   'is_generated': False,
   'alphanum_fraction': 0.2857142857142857,
   'alpha_fraction': 0.25,
   'num_lines': 8,
   'avg_line_length': 9.5,
   'max_line_length': 18},
  'pocke__rubocop-typed.hf.json'),
 ({'blob_id': 'ad6e41590cde9e69f22185d85413ad8fcae8fc1e',
   'path': 'theme-minimal/src/main/resources/META-INF/keycloak-themes.json',
   'content_id': '5d60059e21ecf760854903743ed73ceb256f91bea09c9313bfe3f5a0f94eec31',
   'language': 'JSON',
   'length_bytes': 213,
   'detected_licenses': [],
   'license_type': '',
   'src_encoding': 'utf-8',
   'is_vendor': False,
   'is_generated': False,
   'alphanum_fraction': 0.26291079812206575,
   'alpha_fraction': 0.26291

In [3]:
# read first
mirrors_dir = Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/mirror/chunk-00")

## Print files with alphanumeric < 0.3

In [ ]:
local_filename, full_filename = files[0]
local_filename = local_filename["path"]
archive_name = Path(Path(full_filename).stem).stem + ".tar.gz"

import tarfile
import tempfile

with tempfile.TemporaryDirectory() as temp_dir:
    with tarfile.open(mirrors_dir / archive_name, "r:gz") as tar:
        tar.extractall(temp_dir)

    parent_dir = list(Path(temp_dir).glob("*"))[0]

    with open(Path(temp_dir) / parent_dir / local_filename) as f:
        print(f.read())

a = {a: { b: 1 } }
p a[:a][:b]

b = {a: [{b: 1}]}
p b[:a][0][:b]

c = nil
c[:a][:b]



/tmp/ipykernel_3571745/3949653733.py:10: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(temp_dir)


Many special characters

In [ ]:
files = []
for fp in Path("/home/george/coding_parser/out/chunk-00").glob("*.hf.json"):
    d = orjson.loads(fp.read_bytes())
    for f in d["files"]:
        if f["num_lines"] > 1 and f["alphanum_fraction"] > 0.9:
            files.append((f, fp.name))
        if len(files) == 10:
            break

In [ ]:
# use biome formatter
biome_supported_languages = {"JavaScript", "TypeScript", "JSX", "TSX", "JSON", "HTML", "CSS", "GraphQL"}

In [ ]:
pd.Series(np.concatenate(df["files"].apply(lambda x: [f["language"] for f in x]).values)).value_counts()

JavaScript            348108
Python                184278
Java                  142114
C                     127754
C#                    114830
Markdown              112575
JSON                   92495
PHP                    87010
C++                    69617
TypeScript             68827
Go                     55816
YAML                   39097
Ruby                   35053
HTML                   32848
Shell                  15280
Swift                  14167
Kotlin                 13668
Rust                    8341
reStructuredText        8156
Java Properties         7514
CMake                   6161
Makefile                5228
SQL                     5063
Gradle                  4679
Maven POM               4127
Lua                     4118
Text                    3639
Dockerfile              2671
TOML                    2117
INI                     1717
Go Module               1053
M4Sugar                  556
JSON with Comments       344
Groovy                   228
R             

In [ ]:
local_filename, full_filename = files[0]
local_filename = local_filename["path"]
archive_name = Path(Path(full_filename).stem).stem + ".tar.gz"

import tarfile
import tempfile

with tempfile.TemporaryDirectory() as temp_dir:
    with tarfile.open(mirrors_dir / archive_name, "r:gz") as tar:
        tar.extractall(temp_dir)

    parent_dir = list(Path(temp_dir).glob("*"))[0]

    with open(Path(temp_dir) / parent_dir / local_filename) as f:
        print(f.read())

module OrganizationsHelper
end



/tmp/ipykernel_3571745/3949653733.py:10: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(temp_dir)


Mostly text, usually config files

In [ ]:
df["files"] = df["files"].apply(
    lambda x: [f for f in x if 0.3 < f["alphanum_fraction"] < 0.895 and f["avg_line_length"] > 8]
)

In [ ]:
df = df[df.apply(lambda x: len(x["files"]) > 1, axis=1)]

In [ ]:
df["files"].apply(lambda x: sum([f["num_lines"] * f["avg_line_length"] for f in x])).sum()

np.float64(10457583818.0)

In [ ]:
# 2.6B tokens
10457583818.0 / 4

2614395954.5

In [ ]:
# we want 60% of such data in resulting mix, but 2B tokens for sample used for tokenization
# so 1.2B tokens for now


df.shape

(21381, 19)

In [73]:
df.iloc[:9835]

,repo_name,repo_url,snapshot_id,revision_id,directory_id,branch_name,visit_date,revision_date,committer_date,github_id,star_events_count,fork_events_count,gha_license_id,gha_created_at,gha_updated_at,gha_pushed_at,gha_language,files,num_files
0,revolut-mobile/RxData,https://github.com/revolut-mobile/RxData,,,,,None,None,None,213083066,52,0,,None,2025-05-28T23:20:21+00:00,2024-05-15T13:46:18+00:00,,[{'blob_id': 'a050a132b475c2b6e96812b2269b995d...,94
1,liuym2018/leetcode,https://github.com/liuym2018/leetcode,,,,,None,None,None,373407586,0,0,,2021-06-03T06:42:50+00:00,2021-10-24T21:08:29+00:00,2021-06-16T19:13:14+00:00,JavaScript,[{'blob_id': '4037105cf6e0f8c2b023673195200ef7...,36
2,jsefiani/pokemon-web-app,https://github.com/jsefiani/pokemon-web-app,,,,,None,None,None,228184207,2,0,,None,2023-03-09T01:43:41+00:00,2020-09-10T23:26:51+00:00,,[{'blob_id': '2a20889eced727fc7529add3eaeba754...,6
3,tjma2001/parse-server-azure-storage-reloaded,https://github.com/tjma2001/parse-server-azure...,,,,,None,None,None,179142068,1,0,,None,2023-03-09T03:02:46+00:00,2022-12-09T18:22:38+00:00,,[{'blob_id': '0b85cad07b691b962b687420614de037...,10
4,Krish03na/MerchandiseApp,https://github.com/Krish03na/MerchandiseApp,,,,,None,None,None,247131394,3,0,,None,2024-04-21T09:02:32+00:00,2020-08-04T03:40:45+00:00,,[{'blob_id': '6229aca4414107d9fcb27c9963038104...,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10386,chiyuan666/algorithm,https://github.com/chiyuan666/algorithm,,,,,None,None,None,382570993,1,0,,None,2022-08-27T15:36:26+00:00,2021-07-03T09:08:05+00:00,,[{'blob_id': '8186d2f1cc8f1f089957a6e447d76943...,4
10387,ghettovoice/vuelayers,https://github.com/ghettovoice/vuelayers,,,,,None,None,None,83931926,690,0,,None,2025-08-12T06:27:32+00:00,2023-06-15T07:33:25+00:00,,[{'blob_id': '1906428c272300e66d8e61e31af2c5fa...,188
10388,PastaPastaPasta/dash,https://github.com/PastaPastaPasta/dash,,,,,None,None,None,132179499,1,0,,None,2025-07-16T21:51:33+00:00,2025-08-23T16:14:04+00:00,,[{'blob_id': '870291559259465bd12ea74d73c404c3...,2951
10389,atese3/Licence,https://github.com/atese3/Licence,,,,,None,None,None,218748549,2,0,,None,2022-08-29T20:57:25+00:00,2019-10-31T11:17:04+00:00,,[{'blob_id': '7d6ed97434740c4a467a0ff703e6523d...,8


In [139]:
import subprocess


def is_mostly_hex(text, threshold=0.9):
    if not text:
        return False
    hex_chars = sum(c in "0123456789abcdefABCDEF" for c in text)
    mostly_hex = hex_chars / len(text) >= threshold
    if mostly_hex:
        print("this is mostly hex")
    return mostly_hex


def apply_formatting(directory, verbose=False):
    lint_msg = ""
    if any(
        x.suffix in {".js", ".ts", ".jsx", ".tsx", ".json", ".html", ".css", ".graphql"}
        for x in Path(directory).glob("**/*")
    ):
        biome_cmd = f"npx @biomejs/biome check --unsafe --write {str(directory)}"
        biome_output = subprocess.run(biome_cmd, shell=True, capture_output=True).stdout.decode()
        if verbose:
            print(biome_output)
        lint_msg = biome_output
    elif any(x.suffix in {".py", ".ipynb"} for x in Path(directory).glob("**/*")):
        ruff_cmd = f"uvx ruff format --target-version py311 --config /home/george/coding_parser/pyproject.toml {str(directory)}"
        lint_cmd = f"uvx ruff check --fix --unsafe-fixes --target-version py311 --config /home/george/coding_parser/pyproject.toml {str(directory)}"
        ruff_output = subprocess.run(ruff_cmd, shell=True, capture_output=True).stdout.decode()
        lint_output = subprocess.run(lint_cmd, shell=True, capture_output=True).stdout.decode()
        # lint_msg = lint_output.strip("\x1b[1m\x1b[91m")
        if verbose:
            print(ruff_output)
            print(lint_output)
    else:
        if verbose:
            print(f"No formatting needed for {directory}")
    return lint_msg

In [140]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

batch_size = 5000
chunk_size = 100
for batch_i in range(df.shape[0] // batch_size):
    batch_results = {"repo_name": [], "text": []}
    with ThreadPoolExecutor(16) as executor:
        futures = []
        for i in range(batch_i * batch_size, (batch_i + 1) * batch_size, chunk_size):
            df_chunk = df.iloc[i : i + chunk_size]
            future = executor.submit(process_chunk, df_chunk)
            futures.append(future)
        for result in tqdm(as_completed(futures), total=len(futures)):
            batch_results["repo_name"].extend(result["repo_name"])
            batch_results["text"].extend(result["text"])
    break

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [58]:
s = next(df.iloc[:100].iterrows())[1]

In [131]:
df_chunk = df.iloc[400:500, :]

In [136]:
result = process_chunk(df_chunk)

Checked 46 files in 26ms. Fixed 46 files.
Checked 8 files in 36ms. Fixed 8 files.
Checked 4 files in 3ms. Fixed 4 files.
Checked 50 files in 293ms. Fixed 48 files.
Checked 5 files in 50ms. Fixed 5 files.
Checked 7 files in 132ms. Fixed 7 files.
Checked 0 files in 816µs. No fixes applied.
Checked 1 file in 33ms. Fixed 1 file.
Checked 0 files in 764µs. No fixes applied.
Checked 5 files in 8ms. Fixed 5 files.
Checked 21 files in 18ms. Fixed 21 files.
Checked 378 files in 46s. Fixed 361 files.
Checked 24 files in 17s. Fixed 24 files.
Checked 1 file in 7ms. Fixed 1 file.
Checked 6 files in 181ms. Fixed 6 files.
Checked 176 files in 233s. Fixed 175 files.
Checked 11 files in 19ms. Fixed 11 files.
Checked 11 files in 25ms. Fixed 11 files.
Checked 0 files in 1600µs. No fixes applied.
Checked 16 files in 19ms. Fixed 15 files.
Checked 1 file in 120ms. Fixed 1 file.
Checked 6 files in 12ms. Fixed 6 files.
Checked 1 file in 22ms. Fixed 1 file.
Checked 7 files in 9ms. Fixed 7 files.
Checked 1 file 

In [138]:
import tarfile
import tempfile


chunk_dir = Path("/home/george/coding_parser/out/chunk-00")
mirrors_dir = Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/mirror/chunk-00")


def process_chunk(df_chunk: pd.DataFrame):
    result = {"repo_name": [], "text": []}
    for _, item in df_chunk.iterrows():
        repo_name = item["repo_name"]
        files = ast.literal_eval(item["files"])
        local_filenames = {f["path"] for f in files}
        # print(local_filenames)

        file_contents = []
        archive_name = f"{repo_name.replace('/', '__')}.tar.gz"
        path = mirrors_dir / archive_name
        if path.exists():
            with tempfile.TemporaryDirectory() as temp_dir:
                with tarfile.open(path, "r:gz") as tar:
                    tar.extractall(temp_dir, filter="tar")

                parent_dir = list(Path(temp_dir).glob("*"))[0]

                len(list(parent_dir.glob("**/*")))
                exts = set()

                # remove filtered files
                for f in parent_dir.glob("**/*"):
                    try:
                        if f.is_file():
                            local_path = f.relative_to(parent_dir)
                            if str(local_path) not in local_filenames or is_mostly_hex(f.read_text()):
                                f.unlink()
                            else:
                                exts.add(local_path.suffix)
                    except Exception:
                        pass
                        # print(f"Error processing {f}: {e}")

                len(list(parent_dir.glob("**/*")))

                # apply formatting
                if exts & {".js", ".ts", ".tsx", ".py"}:
                    apply_formatting(parent_dir)

                # read remaining files
                for f in parent_dir.glob("**/*"):
                    try:
                        if f.is_file():
                            with open(f) as f:
                                file_contents.append(f.read())
                    except Exception:
                        pass
                        # print(f"Error processing {f}: {e}")
            result["repo_name"].append(repo_name)
            result["text"].append("<|endoftext|>".join(file_contents))
    return result

In [ ]:
with open("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/stack_v2_sample_train.txt", "w") as fopen:
    listout = []
    for i in tqdm(range(9835)):
        listout.extend([t + "<|endoftext|>" for t in process_repo(df.iloc[i])])
        if len(listout) > 1000:
            _ = fopen.write("".join(listout))
            listout = []

 22%|██▏       | 2201/9835 [41:58<56:16,  2.26it/s]  

Error processing /tmp/tmpvzayali7/pasosdeJesus-Mt77-21ad68d/RamDisco.cpp: 'utf-8' codec can't decode byte 0xe1 in position 158: invalid continuation byte
Error processing /tmp/tmpvzayali7/pasosdeJesus-Mt77-21ad68d/comunCasoPrueba.hpp: 'utf-8' codec can't decode byte 0xe1 in position 99: invalid continuation byte
Error processing /tmp/tmpvzayali7/pasosdeJesus-Mt77-21ad68d/doc/conf.sh: 'utf-8' codec can't decode byte 0xe9 in position 23: invalid continuation byte
Error processing /tmp/tmpvzayali7/pasosdeJesus-Mt77-21ad68d/doc/Makefile: 'utf-8' codec can't decode byte 0xfa in position 125: invalid start byte
Error processing /tmp/tmpvzayali7/pasosdeJesus-Mt77-21ad68d/doc/herram/confaux.sh: 'utf-8' codec can't decode byte 0xf3 in position 2914: invalid continuation byte
Error processing <_io.TextIOWrapper name='/tmp/tmpvzayali7/pasosdeJesus-Mt77-21ad68d/RamDisco.cpp' mode='r' encoding='UTF-8'>: 'utf-8' codec can't decode byte 0xe1 in position 158: invalid continuation byte
Error processing

 23%|██▎       | 2277/9835 [43:10<16:11,  7.78it/s]   

Error processing /tmp/tmpc63cx0q6/lojpark-Baekjoon-25fd144/DFS/17825.cpp: 'utf-8' codec can't decode byte 0xc1 in position 557: invalid start byte
Error processing /tmp/tmpc63cx0q6/lojpark-Baekjoon-25fd144/Dynamic programming/2011.cpp: 'utf-8' codec can't decode byte 0xb9 in position 11: invalid start byte
Error processing /tmp/tmpc63cx0q6/lojpark-Baekjoon-25fd144/Dynamic programming/2133.cpp: 'utf-8' codec can't decode byte 0xb1 in position 16: invalid start byte
Error processing /tmp/tmpc63cx0q6/lojpark-Baekjoon-25fd144/Dynamic programming/11066.cpp: 'utf-8' codec can't decode byte 0xb1 in position 39: invalid start byte
Error processing /tmp/tmpc63cx0q6/lojpark-Baekjoon-25fd144/Dynamic programming/9095.cpp: 'utf-8' codec can't decode byte 0xc0 in position 17: invalid start byte
Error processing /tmp/tmpc63cx0q6/lojpark-Baekjoon-25fd144/Dynamic programming/2579.cpp: 'utf-8' codec can't decode byte 0xb9 in position 13: invalid start byte
Error processing /tmp/tmpc63cx0q6/lojpark-Baekj

 23%|██▎       | 2296/9835 [43:15<47:11,  2.66it/s]

...

 70%|███████   | 6894/9835 [2:22:44<2:53:26,  3.54s/it]

Error processing <_io.TextIOWrapper name='/tmp/tmpubxeqpbd/pichotg-Encheres-a29b9a1/src/bo/EtatVente.java' mode='r' encoding='UTF-8'>: 'utf-8' codec can't decode byte 0xe9 in position 96: invalid continuation byte
Error processing <_io.TextIOWrapper name='/tmp/tmpubxeqpbd/pichotg-Encheres-a29b9a1/src/servlets/multilingue/Langue_fr.properties' mode='r' encoding='UTF-8'>: 'utf-8' codec can't decode byte 0xe8 in position 168: invalid continuation byte
Error processing <_io.TextIOWrapper name='/tmp/tmpubxeqpbd/pichotg-Encheres-a29b9a1/src/servlets/multilingue/Langue_en.properties' mode='r' encoding='UTF-8'>: 'utf-8' codec can't decode byte 0xe8 in position 454: invalid continuation byte


 70%|███████   | 6913/9835 [2:22:47<07:37,  6.39it/s]  

Error processing /tmp/tmpnyo3_4qx/ksh030506-C_Phone-number-manager-2871c4c/index.c: 'utf-8' codec can't decode byte 0xb5 in position 40: invalid start byte
Error processing <_io.TextIOWrapper name='/tmp/tmpnyo3_4qx/ksh030506-C_Phone-number-manager-2871c4c/index.c' mode='r' encoding='UTF-8'>: 'utf-8' codec can't decode byte 0xb5 in position 40: invalid start byte


 70%|███████   | 6924/9835 [2:22:49<10:27,  4.64it/s]

In [3]:
!ls /mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/ -l

total 5139352
drwxrwxr-x 6 george george       4096 Aug 30 10:40 mirror
-rw-rw-r-- 1 george george 5208779856 Aug 30 19:54 stack_v2_sample_train.txt
-rw-rw-r-- 1 george george   53904717 Aug 30 19:56 stack_v2_sample_val.txt


In [ ]:
with open("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/stack_v2_sample_val.txt", "w") as fopen:
    listout = []
    for i in tqdm(range(9835, 10000)):
        listout.extend([t + "<|endoftext|>" for t in process_repo(df.iloc[i])])
        if len(listout) > 1000:
            _ = fopen.write("".join(listout))
            listout = []

In [21]:
!uv pip install hf_transfer

Resolved 1 package in 90ms                                           
Installed 1 package in 2ms                                  
 + hf-transfer==0.1.9


In [ ]:
for k, v in mapping.items():
    if k == "bigcode_the_stack_v2_updated_smol":
        continue
    if k == "finemath-4plus":
        data = datasets.load_dataset("HuggingFaceTB/finemath", "finemath-4plus", split="train", streaming=True)
    else:
        ds = datasets.load_dataset(str(paths / k), streaming=True)
    n_rows = len(ds)
    # total_chars = sum(ds.map(lambda x: {"length": len(x['text'])})["length"])

    chars_needed = 4 * 2e9 * mapping[k]
    chars_needed_train = chars_needed * 0.99
    chars_needed_val = chars_needed - chars_needed_train

    train_handle = open(f"/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/{k}_train.txt", "w")
    val_handle = open(f"/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/{k}_val.txt", "w")
    listout = []
    i = 0
    chars_processed = 0
    for row in ds["train"]:
        listout.append(row["text"] + "<|endoftext|>")
        if chars_processed <= chars_needed_val:
            if "3b" in k or "8b" in k or "12b" in k:
                batch = 100
            else:
                batch = 1000
            if len(listout) > batch:
                _ = val_handle.write("".join(listout))
                listout = []
        elif chars_needed_val < chars_processed <= chars_needed:
            if len(listout) > 1000:
                _ = train_handle.write("".join(listout))
                listout = []
        elif chars_processed > chars_needed:
            break

        chars_processed += len(row["text"])
        i += 1
    train_handle.close()
    val_handle.close()
    print(k, i)

marin-stackexchange-markdown-20b 179949
marin-arxiv-3b 19617
marin-wikipedia-markdown-8b 70527
marin-datashop-science-qa-12b 88888
finemath-4plus 136482
opc-fineweb-code-corpus 282814
zyda-2-100b 147850


In [1]:
from pathlib import Path

In [ ]:
out = Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/train_sample.txt").open("w")
for p in Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/").glob("*_train.txt"):
    out.write(Path(p).read_text())
out.close()

In [ ]:
import numpy as np
from pathlib import Path

out = Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenized_54770/train_sample.npy")
out_arr = None
for p in Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenized_54770/").glob("*_train.npy"):
    if out_arr is None:
        out_arr = np.load(str(p))
    else:
        out_arr = np.concatenate([out_arr, np.load(str(p))])
np.save(str(out), out_arr)

In [1]:
import pickle
from pathlib import Path

vocab = pickle.loads(
    Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenizer/vocab.pickle").read_bytes()
)

In [2]:
import numpy as np

out_arr = np.load(
    "/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenized_54770/train_sample.npy", mmap_mode="r"
)

In [3]:
document_boundaries = (out_arr == 256).nonzero()[0]

In [4]:
merges = pickle.loads(
    Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenizer/merges.pickle").read_bytes()
)

In [5]:
# special case column, split by ':' substring
# special case 2: force merging of punctuation, by splitting of token that goes before punctuation (in order to learn .get etc)
# special case 3: max 4 words before split, as a heuristic split with max 10 tokens

In [6]:
pre_token_counts = {}


def process_diffs(diffs, word_boundaries):
    to_add = []
    for i, d in enumerate(diffs):
        if d >= 20:
            n_additions = min(d // 10, 11)
            # corresponds to i, i+1 in word_boundaries
            left = word_boundaries[i]
            to_add.extend([left + 10 * i for i in range(1, n_additions + 1)])
    return np.array(to_add)


# document + EOS tokens
whitespace_tokens = np.array([i for i, v in vocab.items() if b":" in v] + [256])

punctuation_tokens = np.array([i for i, v in vocab.items() if b":" in v or b"." in v or b"," in v])
pre_punctuation_boundaries = np.isin(out_arr, punctuation_tokens).nonzero()[0] - 1

word_boundaries = np.isin(out_arr, whitespace_tokens).nonzero()[0]

# filter out pre punctuation boundaries that are covered by word boundaries
pre_punctuation_boundaries = pre_punctuation_boundaries[~np.isin(pre_punctuation_boundaries, word_boundaries)]
word_boundaries = np.concatenate((word_boundaries, pre_punctuation_boundaries))
word_boundaries.sort(kind="mergesort")

# filter by length
diffs = word_boundaries[1:] - word_boundaries[:-1]
max_length = np.percentile(diffs, 99)

diff_boundaries_to_add = process_diffs(diffs, word_boundaries)
diff_boundaries_to_add = diff_boundaries_to_add[~np.isin(diff_boundaries_to_add, word_boundaries)]
word_boundaries = np.concatenate((word_boundaries, diff_boundaries_to_add))
word_boundaries.sort(kind="mergesort")

# which indices are actually EOS boundaries across all boundaries?
document_boundaries_indices = np.isin(word_boundaries, document_boundaries).nonzero()[0]
assert len(document_boundaries_indices) == len(document_boundaries)

In [8]:
from tqdm.auto import tqdm

In [15]:
# when splitting by words we need to keep track of the document pointer, in which case
# we should skip EOS token from adding as a key
doc_boundary_ptr = 0
pre_token_counts = {}
should_change_left = False
for i, right in tqdm(enumerate(word_boundaries), total=len(word_boundaries)):
    if i == 0:
        left = 0
    else:
        if i == document_boundaries_indices[doc_boundary_ptr]:
            # we should skip EOS at the next iteration
            should_change_left = True
            doc_boundary_ptr += 1
    # we really don't need 1 token keys anymore
    if right - left > 1:
        length = right - left
        # truncate by length
        delta_length = max(0, int(length - max_length))
        k = tuple(out_arr[left : right - delta_length].tolist())
        if k not in pre_token_counts:
            pre_token_counts[k] = 1
        else:
            pre_token_counts[k] += 1
    if should_change_left:
        left = right + 1
        should_change_left = False
    else:
        left = right

  1%|          | 2261101/328118461 [00:03<09:31, 570090.08it/s]


KeyboardInterrupt: 

In [11]:
word_boundaries[:100]

array([ 25,  34,  44,  54,  62,  72,  89,  98,  99, 104, 113, 114, 119,
       128, 129, 134, 143, 144, 154, 164, 174, 184, 194, 198, 199, 208,
       218, 228, 238, 248, 257, 268, 281, 297, 305, 318, 321, 334, 337,
       342, 352, 362, 368, 371, 383, 386, 402, 412, 422, 425, 435, 445,
       446, 449, 459, 469, 479, 489, 492, 509, 519, 529, 539, 542, 555,
       567, 578, 589, 599, 609, 617, 627, 637, 639, 644, 661, 671, 681,
       683, 687, 699, 709, 719, 729, 739, 744, 762, 764, 766, 776, 786,
       789, 803, 808, 816, 819, 831, 846, 858, 862])

In [2]:
from sample_efficient_gpt.bpe import pre_tokenize

In [ ]:
pre_token_counts = pre_tokenize(
    bpe.splitter, "/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/stack_v2_sample_val.txt", num_processes=12
)

In [ ]:
pre_token_byte_counts: dict[tuple[bytes], int] = {tuple(k.encode()): v for k, v in pre_token_counts.items()}

In [10]:
!ls /mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenized_54770/ -l

total 9151504
-rw-rw-r-- 1 george george  174715034 Sep  6 15:26 finemath-4plus_train.npy
-rw-rw-r-- 1 george george  201021166 Sep  6 15:25 marin-arxiv-3b_train.npy
-rw-rw-r-- 1 george george  112462274 Sep  6 15:43 marin-datashop-science-qa-12b_train.npy
-rw-rw-r-- 1 george george  199665176 Sep  6 15:26 marin-stackexchange-markdown-20b_train.npy
-rw-rw-r-- 1 george george  149742476 Sep  6 15:41 marin-wikipedia-markdown-8b_train.npy
-rw-rw-r-- 1 george george  287220590 Sep  6 15:28 opc-fineweb-code-corpus_train.npy
-rw-rw-r-- 1 george george 3184028428 Sep  6 15:41 stack_v2_sample_train.npy
-rw-rw-r-- 1 george george 4685551224 Sep  6 16:37 train_sample.npy
-rw-rw-r-- 1 george george  376696976 Sep  6 15:43 zyda-2-100b_train.npy


In [ ]:
import pickle

vocab = pickle.loads(
    Path("/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenizer/vocab.pickle").read_bytes()
)
pre_token_counts_tokenized = bpe.pre_tokenize_resume(
    "/mnt/harddrive/datasets/bigcode_the_stack_v2_updated_smol/tokenized_54770/marin-datashop-science-qa-12b_train.npy",
    vocab,
    superbpe=False,
)

100%|██████████| 38374068/38374068 [00:17<00:00, 2139551.24it/s]


In [18]:
bpe.vocab = vocab

In [19]:
bpe.pre_token_byte_counts = pre_token_counts_tokenized

In [20]:
updated_keys, all_counts, pair_to_pre_tokens, all_updated_pairs = None, None, None, None
(
    (updated_key, new_id),
    pre_token_byte_counts_new,
    new_updated_keys,
    all_counts_updated,
    pair_to_pre_tokens_updated,
    all_updated_pairs_updated,
) = bpe.iter_merge_cached(
    bpe.pre_token_byte_counts,
    updated_keys,
    all_counts,
    pair_to_pre_tokens,
    all_updated_pairs,
)

In [21]:
updated_key

(10, 10)

In [22]:
new_id

54770

In [16]:
len(pre_token_counts_tokenized)

1041453

In [17]:
len(pre_token_byte_counts)

192708

In [4]:
from datasets import load_dataset

ds = load_dataset("thepowerfuldeez/the-stack-v2-train-smol-ids-updated-content", split="train", streaming=True)
for row in ds.take(100):
    if "import numpy" in row["text"]:
        print(row["text"])
        break

#include <eigen3/Eigen/Eigen>
#include <map>
#include <vector>
#include <fstream>

using namespace Eigen;

const int iter = 1000;
const double tol = 1e-10;
void power_eng(const MatrixXd &a, double &pld, std::vector<double> &env)
{
    MatrixXd A = MatrixXd(a);
    VectorXd u = VectorXd::Random(a.rows());
    VectorXd v = VectorXd(u);
    for (size_t j = 1; j <= iter; ++j)
    {
        v = u / u.lpNorm<Infinity>(); //利用无穷范数进行归一化
        u = A * v;
    }
    for (size_t i = 0; i < v.size(); ++i)
    {
        env.push_back(v(i));
    }
    //env = v;
    pld = u.lpNorm<Infinity>();
    /*
      MatrixXd A = MatrixXd(a);
    VectorXd u = VectorXd::Random(a.rows());
    for (size_t j = 1; j <= iter; ++j)
    {
        u = A * u;
        u = u / u.squaredNorm();
       
    }
    for (size_t i = 0; i < u.size(); ++i)
    {
        env.push_back(u(i));
    }
    //env = v;
    auto x = u.transpose()*A*u *(u.transpose()*u).inverse(); //瑞利商
    pld = x(0);
    */
}

void inv_power_eng(const M